In [1]:
from openai import Client, OpenAI
from transformers.utils import get_json_schema

client = OpenAI(
    api_key="sk-nothing",
    base_url="http://localhost:11434/v1/",
)

In [6]:
def wikipedia(query: str) -> str:
    """
    Returns the Wikipedia page summary for a given query

    Args:
        query: the page to search for (default None)
    """
    return ""


params = get_json_schema(wikipedia)

tools = [
    {
        "type": "function",
        "function": {
            "name": "wikipedia",
            "description": "Returns the Wikipedia page summary for a given query",
            "parameters": params,
            "strict": False,
        },
    }
]
response = client.chat.completions.create(
    model="granite3.1-dense:8b-instruct-q4_K_M",
    messages=[
        # {
        #     "role": "system",
        #     "content": "You are a helpful assistant that analyzes Python.",
        # },
        {
            "role": "user",
            "content": "Please list one IBM Research laboratory located in the United States. You should use a tool call.",
        },
    ],
    # max_tokens=300,
    tool_choice="auto",
    # {
    #     "type": "function",
    #     "function": {"name": "wikipedia"},
    # },  # "required", # none or auto
    temperature=0.0,
    logprobs=True,
    top_logprobs=1,
    timeout=30,
    tools=tools,
)

In [7]:
response.choices[0]

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='<tool_call>[{"arguments":{"query": "IBM Research lab in the US"},"tool_call":"wikipedia"}]', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))

In [26]:
response.choices[0].message.content

''

In [8]:
# import ollama
from ollama import chat, Message

response = chat(
    "granite3.1-dense:8b-instruct-q4_K_M",
    messages=[
        Message(
            role="user",
            content="Please list one IBM Research laboratory located in the United States. You should use a tool call.",
        )
    ],
    tools=[wikipedia],  # Actual function reference
)

available_functions = {
    "wikipedia": wikipedia,
}

for tool in response.message.tool_calls or []:
    function_to_call = available_functions.get(tool.function.name)
    if function_to_call:
        print("Function output:", function_to_call(**tool.function.arguments))
    else:
        print("Function not found:", tool.function.name)

Function output: 


In [9]:
response

ChatResponse(model='granite3.1-dense:8b-instruct-q4_K_M', created_at='2025-01-06T00:05:23.969485Z', done=True, done_reason='stop', total_duration=1002644417, load_duration=14084917, prompt_eval_count=120, prompt_eval_duration=328000000, eval_count=26, eval_duration=657000000, message=Message(role='assistant', content='', images=None, tool_calls=[ToolCall(function=Function(name='wikipedia', arguments={'query': 'IBM Research - Almaden'}))]))

In [24]:
from ollama import chat
from pydantic import BaseModel


# Define the schema for the response
class FriendInfo(BaseModel):
    name: str
    age: int
    is_available: bool


class FriendList(BaseModel):
    friends: list[FriendInfo]


# schema = {'type': 'object', 'properties': {'friends': {'type': 'array', 'items': {'type': 'object', 'properties': {'name': {'type': 'string'}, 'age': {'type': 'integer'}, 'is_available': {'type': 'boolean'}}, 'required': ['name', 'age', 'is_available']}}}, 'required': ['friends']}
response = chat(
    model="granite3.1-dense:8b-instruct-q4_K_M",
    messages=[
        {
            "role": "user",
            "content": "I have two friends. The first is Ollama 22 years old busy saving the world, and the second is Alonso 23 years old and wants to hang out. Return a list of friends in JSON format",
        }
    ],
    format=FriendList.model_json_schema(),  # Use Pydantic to generate the schema or format=schema
    options={"temperature": 0},  # Make responses more deterministic
)

# Use Pydantic to validate the response
friends_response = FriendList.model_validate_json(response.message.content)
print(friends_response)

friends=[FriendInfo(name='Ollama', age=22, is_available=False), FriendInfo(name='Alonso', age=23, is_available=True)]
